# First Full LoRA Trial with Transformer

## What We Are Doing

Starting with going through what I've done as well as finishing the task of getting my LoRA-fine-tuned model from Hugging Face and running inference on it (i.e. testing it using the test set). 

Note that you'll need the following files in the working directory of the notebook, all available from the repo,

https://github.com/bballdave025/rwkv-lora/tree/main

```
'following files'
  dwb_rouge_scores.py
  dwb_scoring_utils_rouge.py
  dwb_text_utils_data_rep.py
  samsum-test.csv
  samsum-train.csv
  samsum-validation.csv
  system_info_as_script.py
```

If the conversion has already been done, feel free to just transfer

```
samsum-test.json
samsum-train.json
samsum-validation.json
```

without the `*.csv` files. These files come from https://www.kaggle.com/datasets/nileshmalode1/samsum-dataset-text-summarization

## peft (for LoRA) and FLAN-T5-small for the LLM

I'm following what seems to be a great tutorial from Mehul Gupta,

> https://medium.com/data-science-in-your-pocket/lora-for-fine-tuning-llms-explained-with-codes-and-example-62a7ac5a3578
> 
> https://web.archive.org/web/20240522140323/https://medium.com/data-science-in-your-pocket/lora-for-fine-tuning-llms-explained-with-codes-and-example-62a7ac5a3578

Well, I followed that for my first try. I've been making improvements. (I've been making it so it works.)

## Starting

I'm doing this to prepare creating a LoRA for RWKV ( links [here](#Notes-Looking-Forward-to-LoRA-on-RWKV) ) so as to fine-tune it for Pat's OLECT-LM stuff.

In [1]:
# # Don't need this again
#!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

1721754308_20240723T170508-0600


Output was:

`1721754308_20240723T170508-0600`

## Installation - minimize it unless you need `!pip install` commands

(Feel free to drop down to the [TL;DR](#Install-TL;DR) section.)

### Detailed stuff - minimize it

#### Conda environment file - minimize

My <strike>`environment.yml` </strike> `environment-cpu.yml` file for a cpu (specifically, one run on Windows) will have its contents listed below. It should have everything needed for an install anywhere. To make things consistent for CoLab, I've put the `!pip install`s here. The fact that CoLab disconnects in the middle of jobs and only gives a little time with a GPU makes it (it=CoLab) not worth the trouble, but I like having the installs where I can see them. By the way, the directory should have a `full_environment-cpu.yml`, which includes everything for the environment on Windows.

In [3]:
do_want_to_read_realtime = False

if do_want_to_read_realtime:
    with open("environment-cpu.yml", 'r', encoding='utf-8') as fh:
        while True:
            line = fh.readline()
            if not line:
                break
            ##endof:  if not line
            print(line.replace("\n", ""))
        ##endof:  while True
    ##endof:  with open ... fh
##endof:  if do_want_to_read_realtime

For the CPU, I got the following.

```
###############################################
#  Now used for
#+     'full_environment.yml'
#+   which has everything, including 
#+   platform-specific stuff (Windows, here)
#+ AND
#+     'environment.yml'
#+   a meant-to-be OS-agnostic package list
#======

# @file: environment-cpu.yml
# @since 2024-07-25
## 1721754699_2024-07-23T17:11:39-0600
## IMPORTANT NOTES
##
##  A couple of installations were made from git repos. (These next commands
##+ were the originals - look below for the commands needed for 
##+ reproducibility.)
##     >pip install git+https://github.com/huggingface/peft.git
##     >pip install git+https://github.com/nexplorer-3e/qwqfetch
##
##  The commit info will be important for reproducibility.
##
##-----
##  qwqfetch   for system info
##
##   Resolved https://github.com/nexplorer-3e/qwqfetch \
##       to commit f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00
##   Successfully installed qwqfetch-0.0.0
##
##  !!!!! So, the command should be !!!!!
##
## (Windows CMD)
##>pip install git+https://github.com/huggingface/qwqfetch.git@^
## e7b75070c72a88f0f7926cc6872858a2c5f0090d
##
## (Windows PowerShell)
##>pip install git+https://github.com/huggingface/qwqfetch.git@`
## e7b75070c72a88f0f7926cc6872858a2c5f0090d
##
## (*NIX-like)
##$ pip install git+https://github.com/huggingface/qwqfetch.git@\
##  e7b75070c72a88f0f7926cc6872858a2c5f0090d
##  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
##-----
##  peft: I installed PEFT among other things, but I'm picking out 
##+       stuff relevant to peft. PEFT has LoRA in it.
##
##   Resolved https://github.com/huggingface/peft.git \
##       to commit e7b75070c72a88f0f7926cc6872858a2c5f0090d
##   Successfully built peft
##
##  !!!!! So, the command should be !!!!!
## (Windows CMD)
##>pip install git+https://github.com/huggingface/peft.git@^
## e7b75070c72a88f0f7926cc6872858a2c5f0090d
##
## (Windows PowerShell)
##>pip install git+https://github.com/huggingface/peft.git@`
## e7b75070c72a88f0f7926cc6872858a2c5f0090d
##
## (*NIX-like)
##$ pip install git+https://github.com/huggingface/peft.git@\
##  e7b75070c72a88f0f7926cc6872858a2c5f0090d
##  !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
#
###############################################

name: t5-and-lora
channels:
  - defaults
dependencies:
  - python=3.10.14
  - pip=24.1.2
  - pip:
      - pip==24.1.2
      - accelerate==0.30.1
      - bitsandbytes==0.43.1
      - datasets==2.19.1
      - evaluate==0.4.2
      - huggingface-hub==0.23.2
      - humanfriendly==10.0
      - jupyter==1.0.0
      - nltk==3.8.1
      - psutil==6.0.0
      - py-cpuinfo==9.0.0
      - pylspci==0.4.3
      - rouge-score==0.1.2
      - torch==2.3.0
      - transformers==4.41.1
      - trl==0.8.6
      - wmi==1.5.1
```

#### Possible conda then pip install - recommended

If packages aren't installed - a situation that happens every time with CoLab and that will happen the first time on my CPU-with-Windows-in-the-corner-compy-with-three-screens - this is the suggested way to go.

#### Local Machine

<b>1-local)</b> From a local machine (First time only)

&nbsp;&nbsp;&nbsp;&nbsp;<b>(a)</b> (First time only) Start from a shell. Create a conda environment; I do it from `Conda Prompt (miniconda3)`.
     
        conda create --name t5-and-lora python=3.10.14
        conda activate t5-and-lora
        
        pip install --upgrade pip==24.1.2
          # note that you should use the `--upgrade` whether upgrading or downgrading pip
        
        ## possibly needed
        C:\Users\bballdave025\.conda\envs\t5-and-lora\python.exe -m pip install --upgrade pip==24.1.2
        
        pip install jupyter==1.0.0
        
        jupyter notebook SecondFullLoRA_w_Transformer.ipynb

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(The notebook, and the conda environment for that matter, might have different names.)
     
&nbsp;&nbsp;&nbsp;&nbsp;<b>(b)</b> (First time only) Installations.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>(i)</b> Go to the `!pip install` parts ([Install TL;DR](#Install-TL;DR)), making sure that you set the `do_pip_installs` boolean to `True`.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>(ii)</b> Do the installs.

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;<b>(iii)</b> \[Optional\] Run the `!pip freeze` and copy/paste the results to an info spot for local machine.

<b>2-local)</b> (First and subsequent times) Start from a shell. Get into the working directory of the Jupyter notebook. Activate the conda environment and launch the Jupyter notebook.

     conda activate t5-plus-lora
     jupyter notebook SecondFullLoRA_w_Transformer.ipynb

&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;(The notebook, and the conda environment for that matter, might have different names.)

<b>3-local)</b> (Subsequent times) Do not re-run the `!pip install` commands unless something has gone wrong. Make sure that you set the `do_pip_installs` boolean to `False`.

In [3]:
### Install TL;DR

In [4]:
##  If the installs have been completed - either through conda
##+ or through having previously run this notebook, this boolean
##+ should have a `False` value

do_pip_installs = False
if do_pip_installs:
    !powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_' > this_install_outfile.txt
      # no 2>&1. I want to see errors (and only errors for now)
##endof:  if do_pip_installs

Once you have things ready and a jupyter notebook running, you can do the `!pip install` commands that follow, depending on whether you are in CoLab or on a local machine, and whether or not it's the first time.

In [5]:
if do_pip_installs:
    !C:\Users\bballdave025\.conda\envs\rwkv-lora-dave\python.exe -m pip install --upgrade pip==24.1.2 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [6]:
## Ran into issues with imports; seems some got skipped (?)
#
#if do_pip_installs:
#    !pip install accelerate bitsandbytes evaluate datasets huggingface-hub
#    !pip install humanfriendly nltk py-cpuinfo pylspci rouge-score
#    !pip install tensorflow-cpu torch transformers trl
###endof:  if do_pip_installs

Trying this next one on its own, since it might fail when not on Windows. `wmi` <b>Update:</b> This installs fine on CoLab.

In [7]:
## Putting it down the list
# 
#if do_pip_installs:
#    !pip install wmi
###endof:  if do_pip_installs

In [8]:
if do_pip_installs:
    !pip install accelerate==0.30.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [9]:
if do_pip_installs:
    !pip install bitsandbytes==0.43.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [10]:
if do_pip_installs:
    !pip install datasets==2.19.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [11]:
if do_pip_installs:
    !pip install evaluate==0.4.2 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [12]:
if do_pip_installs:
    !pip install huggingface-hub==0.23.2 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [13]:
if do_pip_installs:
    !pip install humanfriendly==10.0 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [14]:
if do_pip_installs:
    !pip install nltk==3.8.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [15]:
if do_pip_installs:
    !pip install py-cpuinfo==9.0.0 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [16]:
if do_pip_installs:
    !pip install pylspci==0.4.3 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [17]:
if do_pip_installs:
    !pip install rouge-score==0.1.2 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [18]:
if do_pip_installs:
    !pip install tensorflow==2.16.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [19]:
if do_pip_installs:
    !pip install torch==2.3.0 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [20]:
if do_pip_installs:
    !pip install transformers==4.41.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [21]:
if do_pip_installs:
    !pip install trl==0.8.6 >> this_install_outfile.txt
##endof:  if do_pip_installs

In [22]:
if do_pip_installs:
    !pip install wmi==1.5.1 >> this_install_outfile.txt
##endof:  if do_pip_installs

And now, for the installs from GitHub repos

In [23]:
if do_pip_installs:
    !pip install git+https://github.com/huggingface/peft.git@e7b75070c72a88f0f7926cc6872858a2c5f0090d >> this_install_outfile.txt
    # maybe get new commit hash @e7b75070c72a88f0f7926cc6872858a2c5f0090d
##endof:  if do_pip_installs

Output was

```
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/peft.git ^
    'C:\Users\bballdave025\AppData\Local\Temp\pip-req-build-j8w_x47_'
  Running command git rev-parse -q --verify 'sha^e7b75070c72a88f0f7926cc6872858a2c5f0090d'
  Running command git fetch -q https://github.com/huggingface/peft.git e7b75070c72a88f0f7926cc6872858a2c5f0090d
  Running command git checkout -q e7b75070c72a88f0f7926cc6872858a2c5f0090d
```

In [24]:
if do_pip_installs:
    !pip install git+https://github.com/nexplorer-3e/qwqfetch.git@f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00 >> this_install_outfile.txt
    # make sure to get new commit hash @f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00 >> this_install_outfile.txt
##endof:  if do_pip_installs

Output was

```
  Running command git clone --filter=blob:none --quiet https://github.com/nexplorer-3e/qwqfetch.git ^
    'C:\Users\Anast\AppData\Local\Temp\pip-req-build-5z5w4uco'
  Running command git rev-parse -q --verify 'sha^f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00'
  Running command git fetch -q https://github.com/nexplorer-3e/qwqfetch.git f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00
```

In [25]:
do_see_install_outputs = False

if do_see_install_outputs:
    !type this_install_outfile.txt
##endof:  if do_see_install_outputs

## Specifics for Windows CPU Environment - minimize it

In [26]:
# # Don't need this again
#!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`1721754699_20240723T171139-0600`

In [27]:
do_see_current_requirements = False

if do_see_current_requirements:
    !powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'
    !pip freeze
    !pip freeze > requirements_loraflan_wincpu.txt
##endof:  if do_see_current_requirements

For Windows CPU, I got the following from `!pip freeze` (along with the prepended timestamp).

```
1721754699_20240723T171139-0600
absl-py==2.1.0
accelerate==0.30.1
aiohttp==3.9.5
aiosignal==1.3.1
anyio==4.4.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.4.1
astunparse==1.6.3
async-lru==2.0.4
async-timeout==4.0.3
attrs==23.2.0
Babel==2.15.0
beautifulsoup4==4.12.3
bitsandbytes==0.43.1
bleach==6.1.0
cached-property==1.5.2
certifi==2024.7.4
cffi==1.16.0
charset-normalizer==3.3.2
click==8.1.7
colorama==0.4.6
comm==0.2.2
datasets==2.19.1
debugpy==1.8.2
decorator==5.1.1
defusedxml==0.7.1
dill==0.3.8
docstring_parser==0.16
evaluate==0.4.2
exceptiongroup==1.2.2
executing==2.0.1
fastjsonschema==2.20.0
filelock==3.15.3
flatbuffers==24.3.25
fqdn==1.5.1
frozenlist==1.4.1
fsspec==2024.3.1
gast==0.6.0
google-pasta==0.2.0
grpcio==1.65.1
h11==0.14.0
h5py==3.11.0
httpcore==1.0.5
httpx==0.27.0
huggingface-hub==0.23.2
humanfriendly==10.0
idna==3.7
intel-openmp==2021.4.0
ipykernel==6.29.5
ipython==8.26.0
ipywidgets==8.1.3
isoduration==20.11.0
jedi==0.19.1
Jinja2==3.1.4
joblib==1.4.2
json5==0.9.25
jsonpointer==3.0.0
jsonschema==4.23.0
jsonschema-specifications==2023.12.1
jupyter==1.0.0
jupyter-console==6.6.3
jupyter-events==0.10.0
jupyter-lsp==2.2.5
jupyter_client==8.6.2
jupyter_core==5.7.2
jupyter_server==2.14.2
jupyter_server_terminals==0.5.3
jupyterlab==4.2.4
jupyterlab_pygments==0.3.0
jupyterlab_server==2.27.3
jupyterlab_widgets==3.0.11
keras==3.4.1
libclang==18.1.1
Markdown==3.6
markdown-it-py==3.0.0
MarkupSafe==2.1.5
matplotlib-inline==0.1.7
mdurl==0.1.2
mistune==3.0.2
mkl==2021.4.0
ml-dtypes==0.3.2
mpmath==1.3.0
multidict==6.0.5
multiprocess==0.70.16
namex==0.0.8
nbclient==0.10.0
nbconvert==7.16.4
nbformat==5.10.4
nest-asyncio==1.6.0
networkx==3.3
nltk==3.8.1
notebook==7.2.1
notebook_shim==0.2.4
numpy==1.26.4
opt-einsum==3.3.0
optree==0.12.1
overrides==7.7.0
packaging==24.1
pandas==2.2.2
pandocfilters==1.5.1
parso==0.8.4
peft @ git+https://github.com/huggingface/peft.git@e7b75070c72a88f0f7926cc6872858a2c5f0090d
platformdirs==4.2.2
prometheus_client==0.20.0
prompt_toolkit==3.0.47
protobuf==4.25.3
psutil==6.0.0
pure_eval==0.2.3
py-cpuinfo==9.0.0
pyarrow==17.0.0
pyarrow-hotfix==0.6
pycparser==2.22
Pygments==2.18.0
pylspci==0.4.3
pyreadline3==3.4.1
python-dateutil==2.9.0.post0
python-json-logger==2.0.7
pytz==2024.1
pywin32==306
pywinpty==2.0.13
PyYAML==6.0.1
pyzmq==26.0.3
qtconsole==5.5.2
QtPy==2.4.1
qwqfetch @ git+https://github.com/nexplorer-3e/qwqfetch.git@f72d222e2fff5ffea9f4e4b3a203e4c4d9e8cf00
referencing==0.35.1
regex==2024.5.15
requests==2.32.3
rfc3339-validator==0.1.4
rfc3986-validator==0.1.1
rich==13.7.1
rouge_score==0.1.2
rpds-py==0.19.0
safetensors==0.4.3
Send2Trash==1.8.3
shtab==1.7.1
six==1.16.0
sniffio==1.3.1
soupsieve==2.5
stack-data==0.6.3
sympy==1.12.1
tbb==2021.13.0
tensorboard==2.16.2
tensorboard-data-server==0.7.2
tensorflow==2.16.1
tensorflow-intel==2.16.1
tensorflow-io-gcs-filesystem==0.31.0
termcolor==2.4.0
terminado==0.18.1
tinycss2==1.3.0
tokenizers==0.19.1
tomli==2.0.1
torch==2.3.0
tornado==6.4.1
tqdm==4.66.4
traitlets==5.14.3
transformers==4.41.1
trl==0.8.6
types-python-dateutil==2.9.0.20240316
typing_extensions==4.12.2
tyro==0.8.5
tzdata==2024.1
uri-template==1.3.0
urllib3==2.2.2
wcwidth==0.2.13
webcolors==24.6.0
webencodings==0.5.1
websocket-client==1.8.0
Werkzeug==3.0.3
widgetsnbextension==4.0.11
WMI==1.5.1
wrapt==1.16.0
xxhash==3.4.1
yarl==1.9.4
```

## Imports

On a local machine (not on CoLab) you shouldn't need to know about the files in the current working directory, other than as information that might be useful in understanding where we're loading datasets or where the modules I've made live.

Modules: `system_info_as_script.py`, `dwb_rouge_scores.py`, and `dwb_scoring_utils_rouge.py`.

Dataset files: `samsum-train.csv`, `samsum-validation.csv`, and `samsum-test.csv`.

In [28]:
from datasets import load_dataset
import random
from random import randrange
import torch
from transformers import AutoTokenizer, \
                         AutoModelForSeq2SeqLM, \
                         TrainingArguments, \
                         pipeline
from transformers.utils import logging
from peft import LoraConfig, \
                 prepare_model_for_kbit_training, \
                 get_peft_model, \
                 AutoPeftModelForSeq2SeqLM, \
                 AutoPeftModelForCausalLM, \
                 PeftConfig, \
                 PeftModel
from trl import SFTTrainer
from huggingface_hub import login, notebook_login

import nltk
import rouge_score
from rouge_score import rouge_scorer, scoring

import pickle
import pprint
import re
import timeit
from humanfriendly import format_timespan
import os

## my module(s), now just in the working directory as .PY files
import system_info_as_script
import dwb_rouge_scores
from dwb_scoring_utils_rouge import compute_google_rouge_score, \
                                    format_rouge_score_rough, \
                                    print_rouge_scores
from dwb_text_utils_data_rep import wc, \
                                    head, \
                                    first_n_chars_in_file, \
                                    csv2json_bare_builtins, \
                                    csv2json_better_builtins_indent, \
                                    csv2json_overkill_builtins_robust, \
                                    csv2json_pandas_basic

## to hide all the warnings
import warnings

OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\Anast\.conda\envs\t5-and-lora\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.

If there's an error such as

`OSError: [WinError 126] The specified module could not be found. Error loading "C:\Users\bballdave025\.conda\envs\t5-and-lora\lib\site-packages\torch\lib\shm.dll" or one of its dependencies.`
 
try adding

`C:\Users\bballdave025\AppData\Roaming\Python\Library\bin`

to the `Path`. 

(You'll have to close the Jupyter Notebook, exit and restart the terminal/conda prompt, load the conda environment, and try the imports again.)

This path contains a bunch of `mkl` stuff, which is what's missing.

By the way, `shm.dll` does exist at the path listed above.

### To get rid of all warnings

In [ ]:
do_ignore_all_warnings = False

if do_ignore_all_warnings:
    warnings.filterwarnings("ignore")
##endof:  if do_ignore_all_warnings

## Saving stuff to save time

In [ ]:
pickle_filename = "lora_flan_t5_cpu_objects.pkl"
objects_to_pickle = []
objects_to_pickle_var_names = []

#incremental
objects_to_pickle_01 = []
objects_to_pickle_var_names_01 = []

## Load the training, validation, and test dataset along with the LLM and its tokenizer

The LLM will be fine-tuned.

### Converting from CSV to JSON

I won't put in all the docs for my methods. <strike>Actually, maybe I will, since I don't remember how I set them all up.</strike>

In [ ]:
train_csv_fname = "samsum-train.csv"
validate_csv_fname = "samsum-validation.csv"
test_csv_fname = "samsum-test.csv"

## to be written
train_json_fname = "samsum-train.json"
validate_json_fname = "samsum-validation.json"
test_json_fname = "samsum-test.json"

csv2json_better_builtins_indent(train_csv_fname, train_json_fname)
csv2json_better_builtins_indent(validate_csv_fname, validate_json_fname)
csv2json_better_builtins_indent(test_csv_fname, test_csv_fname)

In [ ]:
!dir | findstr "json"

In [ ]:
print("wc (like bash) (word count) for the test files.")
print("CSV")
wc(train_csv_fname)
print("JSON")
wc(train_json_fname)

print()

print("head (like bash) (first 5 lines) for the test files.")
print("CSV")
head(5, train_csv_fname)
print("JSON")
head(5, train_json_fname)

print()

print("First 150 characters of the test files.")
print("CSV")
first_n_chars_in_file(150, train_csv_fname)
print("JSON")
first_n_chars_in_file(150, train_json_fname)

In [ ]:
data_files = {'train':'samsum-train.json', 
              'evaluation':'samsum-validation.json',
              'test':'samsum-test.json'}
dataset = load_dataset('json', data_files=data_files)

model_name = "google/flan-t5-small"

model_load_tic = timeit.default_timer()
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
model_load_toc = timeit.default_timer()

model_load_duration = model_load_toc - model_load_tic

print(f"Loading the original model, {model_name}")
print(f"took {model_load_toc - model_load_tic:0.4f} seconds.")

model_load_time_str = format_timespan(model_load_duration)

print(f"which equates to {model_load_time_str}")

#  Next line makes training faster but a little less accurate
model.config.pretraining_tp = 1

tokenizer_tic = timeit.default_timer()
tokenizer = AutoTokenizer.from_pretrained(model_name, 
                                          trust_remote_code=True)
tokenizer_toc = timeit.default_timer()

tokenizer_duration = tokenizer_toc - tokenizer_tic

print()
print("Getting the original tokenizer")
print(f"took {tokenizer_toc - tokenizer_tic:0.4f} seconds.")

tokenizer_time_str = format_timespan(tokenizer_duration)

print(f"which equates to {tokenizer_time_str}")

#  padding instructions for the tokenizer
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

### Saving the model name

In [ ]:
objects_to_pickle.append(model_name)
objects_to_pickle_var_names.append('model_name')

#incremental
objects_to_pickle_01.append(model_name)
objects_to_pickle_var_names_01.append('model_name')

### Trying some things I've been learning (architecture)

In [ ]:
print(model)

In [ ]:
model_arch_str = str(model)

with open("google_-flan-t5-small.model-architecture.txt", 'w', encoding='utf-8') as fh:
    fh.write(model_arch_str)
##endof:  with open ... fh

#### Some other saves

In [ ]:
objects_to_pickle.append(model_arch_str)
objects_to_pickle_var_names.append('model_arch_str')

#incremental
objects_to_pickle_01.append(model_arch_str)
objects_to_pickle_var_names_01.append('model_arch_str')

## Prompt

In [ ]:
def prompt_instruction_format(sample):
    return f""" Instruction:
      Use the Task below and the Input given to write the Response:

      ### Task:
      Summarize the Input

      ### Input:
      {sample['dialogue']}

      ### Response:
      {sample['summary']}
      """
##endof:  prompt_instruction_format(sample)

## Trainer - the LoRA Setup Part

In [ ]:
# # Don't need this anymore
!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

#### Okay, here are the args and config

In [ ]:
#  some arguments to pass to the trainer
training_args = TrainingArguments(
                    output_dir='output',
                    num_train_epochs=1,
                    per_device_train_batch_size=4,
                    learning_rate=2e-4,
                    do_eval=True,
                    per_device_eval_batch_size=4,
                    eval_strategy='steps',
                    eval_steps=250, # will do 250, then 50
                    hub_model_id=" ",
                    save_strategy='steps',
                    save_steps=250,
                    overwrite_output_dir=True,
                    #log_level='info', # uncomment to see details
                    logging_dir='logging',
                    logging_strategy='steps',
                    logging_first_step=True,
                    logging_steps=250,
)

#  the fine-tuning (peft for LoRA) stuff
peft_config = LoraConfig(lora_alpha=16,
                         lora_dropout=0.1,
                         r=16,
                         bias='none',
                         task_type='CAUSAL_LM',
)

### Details on `TrainingArguments`

Documentation is at:

https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments

archived at

https://web.archive.org/web/20240608164657/https://huggingface.co/docs/transformers/main/en/main_classes/trainer#transformers.TrainingArguments

The actual file (from which the docs come) is at the following path on my computer

`C:\Users\bballdave025\.conda\envs\rwkv-lora-pat\Lib\site-packages\transformers\training_args.py`

### We're going to start timing stuff, so here's some system info

`system_info_as_script.py` is a script I wrote with the help
of a variety of StackOverflow and documentation sources.
It should be in the working directory.

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

In [ ]:
system_info_as_script.run(do_network_info=False)

#### Find specific dialog - minimize

I found a nice, short, interesting conversation while doing the random summaries, so I went and found its index.

In [ ]:
tic = timeit.default_timer()

str_to_find = "Damien: Omg..I'm glad Sunday is only once a week"

for sample_num in range(len(dataset['test'])):
    this_sample = dataset['test'][sample_num]
    this_dialogue = this_sample['dialogue']
    if str_to_find in this_dialogue:
        print(f"sample_num: {sample_num}")
        print(f"this_dialogue: \n{this_dialogue}")
        print()
        print("this_sample:")
        print(str(this_sample))
        print()
    ##endof:  if str_to_find in this_dialogue
##endof:  for sample_number in range(len(dataset))

toc = timeit.default_timer()

print("Finding the sample in the test dataset (well,")
print("actually looking at every sample in the test")
print("dataset, regardless of whether we had found")
print("something.")
print(f"took {toc - tic:0.4f} seconds.")

my_duration = toc - tic

elapsed_time_str = format_timespan(my_duration)

print(f"which equates to {elapsed_time_str}")

print()

print(f"Total size of test dataset: {sample_num}")

The interesting conversation

In [ ]:
my_index = 224
my_complete_entry = dataset['test'][my_index]
my_cool_str = dataset['test'][my_index]['dialogue']
print(my_cool_str)
objects_to_pickle.append(my_cool_str)
objects_to_pickle_var_names.append('my_cool_str')
my_cool_list = [f"my_index: {my_index}", my_cool_str, my_complete_entry]
pprint.pp(my_cool_list)
objects_to_pickle.append(my_cool_list)
objects_to_pickle_var_names.append('my_cool_list')

#incremental
objects_to_pickle_01.append(my_cool_str)
objects_to_pickle_var_names_01.append('my_cool_str')
objects_to_pickle_01.append(my_cool_list)
objects_to_pickle_var_names_01.append('my_cool_list')

### Let's get the sizes of all parts of the dataset

In [ ]:
size_of_train = len(dataset['train'])
size_of_eval = len(dataset['evaluation'])
size_of_test = len(dataset['test'])

print(f"size_of_train : {size_of_train}")
print(f"size_of_eval  : {size_of_eval}")
print(f"size_of_test  : {size_of_test}")

### Try for a baseline (for out-of-the-box, pretrained model)

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

#### Just one summarization to begin with, randomly picked

##### Well, not so randomly, anymore

In [ ]:
summarizer = pipeline('summarization', 
                      model=model, 
                      tokenizer=tokenizer
)

Just one summarization to begin with, randomly picked ... but
now with th possibility of a known seed, to allow visual 
comparison with after-training results.

<strike>I'M NOT GOING TO USE THIS REPEATED SEED, I'm just going to
use the datum at the first index to compare.</strike>

For sharing with Pat, I'm making it repeatable

In [ ]:
do_seed_for_repeatable = True

if do_seed_for_repeatable:
    rand_seed_for_randrange = 137
    random.seed(rand_seed_for_randrange)
##endof:  if do_seed_for_repeatable

sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

res = summarizer(sample["dialogue"])

print(f"flan-t5-small summary:\n{res[0]['summary_text']}")

#### Now, a couple summarizations with comparisons to ground truth

In [ ]:
pred_test_list = []
ref_test_list = []

sample_num = 0

this_sample = dataset['test'][sample_num]

print(f"dialogue: \n{this_sample['dialogue']}\n---------------")

ground_summary = this_sample['summary']
res = summarizer(this_sample['dialogue'])
res_summary = res[0]['summary_text']

print(f"human-genratd summary:\n{ground_summary}")
print(f"flan-t5-small summary:\n{res_summary}")

ref_test_list.append(ground_summary)
pred_test_list.append(res_summary)

#  Yes, I have just one datum, but I'm setting things up to
#+ work well with a later loop, i.e. with lists
results_test_0 = compute_google_rouge_score(
                    predictions=pred_test_list,
                    references=ref_test_list,
                    use_aggregator=False
)

print("Output from 'list(results_test_0.keys())'")
print(list(results_test_0.keys()))
# ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

In [ ]:
print_rouge_scores(results_test_0, 0)

In [ ]:
#  I don't want to aggregate, yet.
pred_test_list = []
ref_test_list = []

sample_num = 224

this_sample = dataset['test'][sample_num]

print(f"dialogue: \n{this_sample['dialogue']}\n---------------")

ground_summary = this_sample['summary']
res = summarizer(this_sample['dialogue'])
res_summary = res[0]['summary_text']

print(f"human-genratd summary:\n{ground_summary}")
print(f"flan-t5-small summary:\n{res_summary}")

ref_test_list.append(ground_summary)
pred_test_list.append(res_summary)

results_test_224 = compute_google_rouge_score(
                      predictions=pred_test_list,
                      references=ref_test_list,
                      use_aggregator=False
)

In [ ]:
print_rouge_scores(results_test_224, 224)

#### Verbosity stuff - get rid of the nice advice

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

In [ ]:
log_verbosity_is_critical = \
  logging.get_verbosity() == logging.CRITICAL # alias FATAL, 50
log_verbosity_is_error = \
  logging.get_verbosity() == logging.ERROR # 40
log_verbosity_is_warn = \
  logging.get_verbosity() == logging.WARNING # alias WARN, 30
log_verbosity_is_info = \
  logging.get_verbosity() == logging.INFO # 20
log_verbosity_is_debug = \
  logging.get_verbosity() == logging.DEBUG # 10

print( "The statement, 'logging verbosity is CRITICAL' " + \
      f"is {log_verbosity_is_critical}")
print( "The statement, 'logging verbosity is    ERROR' " + \
      f"is {log_verbosity_is_error}")
print( "The statement, 'logging verbosity is  WARNING' " + \
      f"is {log_verbosity_is_warn}")
print( "The statement, 'logging verbosity is     INFO' " + \
      f"is {log_verbosity_is_info}")
print( "The statement, 'logging verbosity is    DEBUG' " + \
      f"is {log_verbosity_is_debug}")

print()

init_log_verbosity = logging.get_verbosity()
print(f"The value of logging.get_verbosity() is: {init_log_verbosity}")

print()

init_t_n_a_w = os.environ.get('TRANSFORMERS_NO_ADVISORY_WARNINGS')
print(f"TRANSFORMERS_NO_ADIVSORY_WARNINGS: {init_t_n_a_w}")

### Actual Baseline on Complete Test Set

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y%m%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

<b>!!! NOTE</b> You'd better <b>make 
dang sure you want the lots of output</b> 
before you set this next boolean to `True`

In [ ]:
do_have_lotta_output_from_all_dialogs_summaries_1 = False

# Are you sure about the value of that last boolean? 1

In [ ]:
print("That last boolean has the value:")
print(f"{do_have_lotta_output_from_all_dialogs_summaries_1}")

There could be up to megabytes worth of text output if you've changed it to `True`.

In [ ]:
##  Haven't tried this environment variable, 
##+ because the logging seemed easier,
##+ and the logging worked
#os.environ("TRANSFORMERS_NO_ADVISORY_WARNINGS") = 1

logging.set_verbosity_error()

baseline_sample_dialog_list = []
baseline_prediction_list = []
baseline_reference_list = []

baseline_tic = timeit.default_timer()

for sample_num in range(len(dataset['test'])):
    this_sample = dataset['test'][sample_num]
    
    if do_have_lotta_output_from_all_dialogs_summaries_1:
        print(f"dialogue: \n{this_sample['dialogue']}\n---------------")
    ##endof:  if do_have_lotta_output_from_all_dialogs_summaries_1
    
    ground_summary = this_sample['summary']
    res = summarizer(this_sample['dialogue'])
    res_summary = res[0]['summary_text']
    
    if do_have_lotta_output_from_all_dialogs_summaries_1:
        print(f"human-genratd summary:\n{ground_summary}")
        print(f"flan-t5-small summary:\n{res_summary}")
    ##endof:  if do_have_lotta_output_from_all_dialogs_summaries_1
    
    baseline_sample_dialog_list.append(this_sample['dialogue'])
    baseline_reference_list.append(ground_summary)
    baseline_prediction_list.append(res_summary)
##endof:  for sample_num in range(len(dataset['test']))

baseline_toc = timeit.default_timer()

baseline_duration = baseline_toc - baseline_tic

print( "Getting things ready for scoring (doing the baseline)")
print(f"took {baseline_toc - baseline_tic:0.4f} seconds.")

baseline_time_str = format_timespan(baseline_duration)

print(f"which equates to {baseline_time_str}")

rouge = load_metric('rouge', trust_remote_code=True)

baseline_results = compute_google_rouge_score(
                      predictions=baseline_prediction_list,
                      references=baseline_reference_list,
                      use_aggregator=True
)

# >>> print(list(baseline_results.keys()))
# ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

objects_to_pickle.append(baseline_sample_dialog_list)
objects_to_pickle_var_names.append('baseline_sample_dialog_list')
objects_to_pickle.append(baseline_prediction_list)
objects_to_pickle_var_names.append('baseline_prediction_list')
objects_to_pickle.append(baseline_reference_list)
objects_to_pickle_var_names.append('baseline_reference_list')
objects_to_pickle.append(baseline_results)
objects_to_pickle_var_names.append('baseline_results')

#incremental
objects_to_pickle_01.append(baseline_sample_dialog_list)
objects_to_pickle_var_names_01.append('baseline_sample_dialog_list')
objects_to_pickle_01.append(baseline_prediction_list)
objects_to_pickle_var_names_01.append('baseline_prediction_list')
objects_to_pickle_01.append(baseline_reference_list)
objects_to_pickle_var_names_01.append('baseline_reference_list')
objects_to_pickle_01.append(baseline_results)
objects_to_pickle_var_names_01.append('baseline_results')

In [ ]:
##  Haven't tried this environment variable, 
##+ because the logging seemed easier,
##+ and the logging worked
# os.environ("TRANSFORMERS_NO_ADVISORY_WARNINGS") = init_t_n_a_w

logging.set_verbosity(init_log_verbosity)

In [ ]:
print_rouge_scores(baseline_results, "BASELINE")

## A Saving Checkpoint - 1

In [ ]:
#incremental
pickle_filename_before_training = "lora_flan_t5_cpu_objects_-_01_before_training.pkl"
pickle_filename_01 = pickle_filename_before_training

objects_to_pickle_var_names_01.append('objects_to_pickle_var_names_01')
objects_to_pickle_01.append(objects_to_pickle_var_names_01)

with open(pickle_filename_01, 'wb') as pfh:
    pickle.dump(objects_to_pickle_01 , pfh)
##endof:  with open ... as pfh # (pickle file handle)

pickle_filename_02 = "lora_flan_t5_cpu_objects_02.pkl" # will likely change
objects_to_pickle_02 = []
objects_to_pickle_var_names_02 = []

### Trainer - the Actual Trainer Part

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

In [ ]:
trainer = SFTTrainer( model=model,
                      train_dataset=dataset['train'],
                      eval_dataset=dataset['evaluation'],
                      peft_config=peft_config,
                      tokenizer=tokenizer,
                      packing=True,
                      formatting_func=prompt_instruction_format,
                      args=training_args,
                      max_seq_length=750
)
##  Warnings are below output.

<hr/>

## Let's Train This LoRA Thing and See How It Does!

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

### The long-time-taking training code is just below.

In [ ]:
tic = timeit.default_timer()
trainer.train()
toc = timeit.default_timer()
print(f"tic: {tic}")
print(f"toc: {toc}")
training_duration = toc - tic
print(f"Training took {toc - tic:0.4f} seconds.")
training_time_str = format_timespan(training_duration)
print(f"which equates to {training_time_str}")

#### The training stats - CPU

(well, mostly just loss on training and validation sets) were

##### When I checkpointed every 50 steps

the values were

```
    Step  | Training Loss | Validation Loss
  --------+---------------+------------------
    50    |  1.830600     |  0.372227
    100   |  0.448100     |  0.126734
    150   |  0.251900     |  0.089366
    200   |  0.202500     |  0.070181
    250   |  0.173600     |  0.059308
!!  ^^^      ^^^^^^^^        ^^^^^^^^
    300   |       |  
    350   |       |  
    400   |       |  
    450   |       |  
    500   |       |  
!!  ^^^      ^^^^^^^^        ^^^^^^^^
    550   |       |  
    600   |       |  
    650   |       |  
    700   |       |  
    750   |       |  
!!  ^^^      ^^^^^^^^        ^^^^^^^^
    800   |       |  
    850   |       |  
    900   |       |  
    950   |       |  
    1000  |       |  
!!  ^^^^     ^^^^^^^^        ^^^^^^^^
```

##### When I checkpointed every 250 steps 

the values were


```
                                               | Compare | 50-step  | 50-step
    Step  | Training Loss | Validation Loss    | 50-step | T.L.     | V.L.
  --------+---------------+------------------==+==-------+----------+----------
    250   |               |                    |         |          |           
    500   |               |                    |         |          |
    750   |               |                    |         |          |
   1000   |               |                    |         |          |
```

<b>N.B.</b> I don't know why it always stops at 1000 - I could look in the docs for a parameter, but I think we get the idea we want.
Oh, I think that one step has 4 samples from the dataset, which would make things match up. Still, with the epoch run, I had 1536 steps.


##### When I only checkpointed for the epoch

the values were

```
  Training Loss | Epoch | Step | Validation Loss
 ---------------+-------+------+-----------------
      0.0685    |  1.0  | 1536 |     0.0226
```

(I only have this for the CPU.)

In [ ]:
objects_to_pickle.append(tokenizer)
objects_to_pickle_var_names.append('tokenizer')
objects_to_pickle.append(trainer)
objects_to_pickle_var_names.append('trainer')

#incremental
objects_to_pickle_02.append(tokenizer)
objects_to_pickle_var_names_02.append('tokenizer')
objects_to_pickle_02.append(trainer)
objects_to_pickle_var_names_02.append('trainer')

## A Saving Checkpoint - 2

In [ ]:
#incremental
pickle_filename_after_training = "lora_flan_t5_cpu_objects_-_02_after_training.pkl"
pickle_filename_02 = pickle_filename_before_training

objects_to_pickle_var_names_02.append('objects_to_pickle_var_names_02')
objects_to_pickle_02.append(objects_to_pickle_var_names_02)

with open(pickle_filename_02, 'wb') as pfh:
    pickle.dump(objects_to_pickle_02 , pfh)
##endof:  with open ... as pfh # (pickle file handle)

pickle_filename_03 = "lora_flan_t5_cpu_objects_03.pkl" # will likely change
objects_to_pickle_03 = []
objects_to_pickle_var_names_03 = []

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

## Save the Trainer to Hugging Face and Get Our Updated Model

### Commenting out the login to HuggingFace stuff.

In [ ]:
## # Don't need this again
#!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

I'm following the [(archived) tutorial from Mehul Gupta on Medium](https://web.archive.org/web/20240522140323/https://medium.com/data-science-in-your-pocket/lora-for-fine-tuning-llms-explained-with-codes-and-example-62a7ac5a3578); since it's archived, you can follow exactly what I'm doing.

Running this next line of code will come up with a dialog box with text entry,
and I'm now using the `@thebballdave025` for Hugging Face stuff.

<b>Make sure to use the WRITE token, here.</b>

In [ ]:
##  This will come up with a dialog box with text entry.
##+ and I'm now using @thebballdave025 for Hugging Face.
#
## Use the write token, here.
#notebook_login()

In [ ]:
## Save tokenizer and create a tokenizer model card
##taking out#tokenizer.save_pretrained('testing')
#  #  'testing' is the local directory
#
## Create the trainer model card
#trainer.create_model_card()
#
## Push the results to the Hugging Face Hub
#trainer.push_to_hub()

## Actually Get the Model from Hugging Face

### Commenting out stuff from Hugging Face login

Running this next line of code will come up with a dialog box with text entry,
and I'm now using the `@thebballdave025` for Hugging Face stuff.

<b>Make sure to use the READ token, here.</b>

In [ ]:
## Read token. Will bring up text entry to paste token string
#notebook_login()

In [ ]:
## # Don't need this again
#!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

(If you have problems that note `data_files` or `dataset` or `prompt_instruction_format`, make sure that the cells where these are defined have been run, i.e. the kernel hasn't been restarted since they were initialized.)

### Commenting out - Hugging Face Stuff from Login

In [ ]:
## My trained model from Hugging Face
#
#new_model_name = "thebballdave025/dwb-flan-t5-small-lora-finetune-2"

In [ ]:
#new_model_load_tic = timeit.default_timer()
#new_model = AutoModelForSeq2SeqLM.from_pretrained(new_model_name)
#new_model_load_toc = timeit.default_timer()
#
#new_model_load_duration = new_model_load_toc - new_model_load_tic
#
#print(f"Loading the LoRA-fine-tuned model, {new_model_name}")
#print(f"took {new_model_load_toc - new_model_load_tic:0.4f} seconds.")
#
#new_model_load_time_str = format_timespan(new_model_load_duration)
#
#print(f"which equates to {new_model_load_time_str}")
##
##  Next line makes training faster but a little less accurate
##+ but I'm not training it down here.
##new_model.config.pretraining_tp = 1

### New instead of commenting out - From Local Machine

Starting the hopefully-fixed rest of it, now.

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

In [ ]:
base_model_name = model_name ## = "google/flan-t5-small"
adapter_model_name = "output/checkpoint-1536" # the path to the adapter

In [ ]:
new_model = AutoModelForSeq2SeqLM.from_pretrained(base_model_name)
# Below, note that `new_model` is a parameter. We're changing it.
new_model = AutoPeftModelForCausalLM.from_pretrained(new_model, adapter_model_name)

In [ ]:
#  I'm saving it, but I don't know how important that will be.
#+ I do know I won't use it in this notebook.
#
#  Reference:
# ref="https://web.archive.org/web/20240628145441/" + \
#     "https://huggingface.co/docs/trl/main/en/use_model"
#> You can also merge the adapters into the base model 
#> so you can use the model like a normal transformers model, 
#> however the checkpoint will be significantly bigger
#
do_save_merged = False

if do_save_merged:
    new_model_2_save = new_model.merge_and_unload()
    new_model_2_save.save_pretrained("merged_adapters")
##endof:  if do_save_merged

In [ ]:
same_tokenizer = AutoTokenizer.from_pretrained(base_model_name)

#### Stuff for model architecture - post-LoRA

In [ ]:
print(new_model)

In [ ]:
new_model_arch_str = str(new_model)

with open(
  "dwb-flan-t5-small-w-lora-finetune-2.model-architecture.txt", 
  'w', 
  encoding='utf-8') as fhn:
    fhn.write(new_model_arch_str)
##endof:  with open ... fhn

objects_to_pickle.append(base_model_name)
objects_to_pickle_var_names.append('base_model_name')
objects_to_pickle.append(adapter_model_name)
objects_to_pickle_var_names.append('adapter_model_name')

objects_to_pickle_03.append(base_model_name)
objects_to_pickle_var_names_03.append('base_model_name')
objects_to_pickle_03.append(adapter_model_name)
objects_to_pickle_var_names_03.append('adapter_model_name')

objects_to_pickle.append(new_model_arch_str)
objects_to_pickle_var_names.append('new_model_arch_str')

objects_to_pickle_03.append(new_model_arch_st)
objects_to_pickle_var_names_03.append('new_model_arch_str')

@todo : get some Python version of `diff` going on here. I'm just using Cygwin/bash to see the LoRA additions.

### Let's start by doing the single-dialogue summaries we used before.

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

#### Note on the `tokenizer` parameter in the `pipline` function - minimize

Note that in Guptal's example/tutorial, he did not give a `tokenizer` parameter to the `summarizer = pipeline` code. I haven't been able to get that working.

#### Consistency in values of model and tokenizer

In [ ]:
# If we want to keep it consistent, use these. If not, change them at will.
model_to_use = new_model
tokenizer_to_use = same_tokenizer

#### Try one picked at random

##### Well, not so randomly, anymore

In [ ]:
summarizer_tat = pipeline('summarization', 
                          model=model_to_use,
                          tokenizer=tokenizer_to_use
)

There might be a warning with `The model 'PeftModelForCausalLM' is not supported for summarization. Supported models are` `...`. This is just about things not being officially integrated into the Transformer library.

(
  Full warning, for reference:

    The model 'PeftModelForCausalLM' is not supported for summarization. Supported models are ['BartForConditionalGeneration', 'BigBirdPegasusForConditionalGeneration', 'BlenderbotForConditionalGeneration', 'BlenderbotSmallForConditionalGeneration', 'EncoderDecoderModel', 'FSMTForConditionalGeneration', 'GPTSanJapaneseForConditionalGeneration', 'LEDForConditionalGeneration', 'LongT5ForConditionalGeneration', 'M2M100ForConditionalGeneration', 'MarianMTModel', 'MBartForConditionalGeneration', 'MT5ForConditionalGeneration', 'MvpForConditionalGeneration', 'NllbMoeForConditionalGeneration', 'PegasusForConditionalGeneration', 'PegasusXForConditionalGeneration', 'PLBartForConditionalGeneration', 'ProphetNetForConditionalGeneration', 'SeamlessM4TForTextToText', 'SeamlessM4Tv2ForTextToText', 'SwitchTransformersForConditionalGeneration', 'T5ForConditionalGeneration', 'UMT5ForConditionalGeneration', 'XLMProphetNetForConditionalGeneration'].

)

If the warning is there, it can safely be ignored. 

### (You don't need to spend 2 weeks trying to fix it, believing that all the warnings had been suppressed so it must be an error. You definitely shouldn't do that.)

cf. https://web.archive.org/web/20240722220713/https://huggingface.co/tiiuae/falcon-40b/discussions/8

> Sorry about the delay, the `The model 'RWForCausalLM'`

ours says `The model 'PeftModelForCausalLM'`

> `is not supported for text-generation.`

ours says `is not supported for summarization.`

> comes from the model not being integrated into the
> core part of the transformers library yet. It's just
> a warning, and generation should follow afterwards.

hopefully this can be amended with "and summarization should follow afterwards."

##### OR

https://web.archive.org/web/20240722220840/https://gist.github.com/ahoho/ba41c42984faf64bf4302b2b1cd7e0ce

> I believe that is a just warning that you can safely ignore.
> For the versions of transformers & PEFT I was using
> (4.28.1 and 0.3.0.dev0, respectively), `PeftModelForCausalLM`

(Our same model name)

> had not been added to the text-generation

(Hopefully everything is consistent if we replace 'text-generation' with 'summarization'.)

> pipelines list of supported models (but, as you can see,
> the underlying `LlamaForCausalLM`

(For us, replace `LlamaForCausalLM` with `T5ForConditionalGeneration`)

> upon which the Peft model
> is added is supported--i.e., the warning is spurious)

Just one summarization to begin with, randomly picked ... but
now with th possibility of a known seed, to allow visual 
comparison with after-training results.

<strike>I'M NOT GOING TO USE THIS REPEATED SEED, I'm just going to
use the datum at the first index to compare.</strike>

User repeatability when sharing with Pat

In [ ]:
do_seed_for_repeatable = True

if do_seed_for_repeatable:
    rand_seed_for_randrange = 137
    random.seed(rand_seed_for_randrange)
##endof:  if do_seed_for_repeatable

sample = dataset['test'][randrange(len(dataset["test"]))]
print(f"dialogue: \n{sample['dialogue']}\n---------------")

res = summarizer_tat(sample["dialogue"])

print(f"dwb-flan-t5-small-w-lora-finetune-2 summary:\n{res[0]['summary_text']}")

#### Notes on investigations to fix weirdness

@todo : format this more nicely

```
## Trials to fix weirdness.
## model=old_model, tokenizer=old_tokenizer : matches baseline 
##                                            (quick)
## model=new_model, tokenizer=new_tokenizer : weird results
##                                            (takes significantly longer, too)
## model=new_model, tokenizer=old_tokenizer : weird results
##                                            (takes significantly longer, too)
## model=old_model, tokenizer=new_tokenizer : actually matches baseline, which
##                                            would seem to require a change in
##                                            hypothesis as to why the
##                                            weirdness and longer inference are
##                                            happening. (Likely not tokenizer.)
##                                            (quick)
##
##  I had thought that doing 'old_model' and 'new_tokenizer' gave me weird
##+ results, too. Good thing to come back and check things.
##  Still, the training results show the model was learning and improving.
```

#### Now, a couple summarizations with comparison to ground truth

In [ ]:
pred_test_list = []
ref_test_list = []

sample_num = 0

this_sample = dataset['test'][sample_num]

print(f"dialogue: \n{this_sample['dialogue']}\n---------------")

ground_summary = this_sample['summary']
res = summarizer_tat(this_sample['dialogue'])
res_summary = res[0]['summary_text']

print(f"human-genratd summary:\n{ground_summary}")
print(f"dwb-flan-t5-small-lora-finetune-2 summary:\n{res_summary}")

ref_test_list.append(ground_summary)
pred_test_list.append(res_summary)

#  Yes, I have just one datum, but I'm setting things up to
#+ work well with a loop (meaning lists for pred and ref).
results_test_0 = compute_google_rouge_score(
                            predictions=pred_test_list,
                            references=ref_test_list,
                            use_aggregator=False
)

# >>> print(list(results_test.keys()))
# ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

In [ ]:
print_rouge_scores(results_test_0, 0)

In [ ]:
# I don't want to aggregate, yet
pred_test_list = []
ref_test_list = []

sample_num = 224

this_sample = dataset['test'][sample_num]

print(f"dialogue: \n{this_sample['dialogue']}\n---------------")

ground_summary = this_sample['summary']
res = summarizer_tat(this_sample['dialogue'])
res_summary = res[0]['summary_text']

print(f"human-genratd summary:\n{ground_summary}")
print(f"dwb-flan-t5-small-lora-finetune-2:\n{res_summary}")

ref_test_list.append(ground_summary)
pred_test_list.append(res_summary)

rouge = load_metric('rouge', trust_remote_code=True)

results_test_224 = compute_google_rouge_score(
                     predictions=pred_test_list,
                     references=ref_test_list,
                     use_aggregator=False
)

In [ ]:
print_rouge_scores(results_test_224, 224)

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

## Evaluation on the Test Set and Comparison to Baseline

#### Verbosity stuff - get rid of the nice advice

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

In [ ]:
log_verbosity_is_critical = \
  logging.get_verbosity() == logging.CRITICAL # alias FATAL, 50
log_verbosity_is_error = \
  logging.get_verbosity() == logging.ERROR # 40
log_verbosity_is_warn = \
  logging.get_verbosity() == logging.WARNING # alias WARN, 30
log_verbosity_is_info = \
  logging.get_verbosity() == logging.INFO # 20
log_verbosity_is_debug = \
  logging.get_verbosity() == logging.DEBUG # 10

print( "The statement, 'logging verbosity is CRITICAL' " + \
      f"is {log_verbosity_is_critical}")
print( "The statement, 'logging verbosity is    ERROR' " + \
      f"is {log_verbosity_is_error}")
print( "The statement, 'logging verbosity is  WARNING' " + \
      f"is {log_verbosity_is_warn}")
print( "The statement, 'logging verbosity is     INFO' " + \
      f"is {log_verbosity_is_info}")
print( "The statement, 'logging verbosity is    DEBUG' " + \
      f"is {log_verbosity_is_debug}")

print()

init_log_verbosity = logging.get_verbosity()
print(f"The value of logging.get_verbosity() is: {init_log_verbosity}")

print()

init_t_n_a_w = os.environ.get('TRANSFORMERS_NO_ADVISORY_WARNINGS')
print(f"TRANSFORMERS_NO_ADIVSORY_WARNINGS: {init_t_n_a_w}")

### Here's the actual evaluation

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

<b>!!! NOTE !!!</b> I'm going to use `tat` (with an underscore
or undescores before, after, or surrounding the variable names)
to indicate 'testing-after-training'.

I guess I could have used `inference`, but I didn't.

<b>!!! another NOTE</b> You'd better <b>make 
dang sure you want the lots of output</b> 
before you set this next boolean to `True`

In [ ]:
do_have_lotta_output_from_all_dialogs_summaries_2 = False

# Are you sure about the value of that last boolean? 2

In [ ]:
print("That last boolean has the value:")
print(f"{do_have_lotta_output_from_all_dialogs_summaries_2}")

There could be up to megabytes worth of text output if you've changed it to `True`.

In [ ]:
logging.set_verbosity_error()

tat_summarizer = summarizer_tat

tat_sample_dialog_list = []
prediction_tat_list = []
reference_tat_list = []

tat_tic = timeit.default_timer()

for sample_num in range(len(dataset['test'])):
    this_sample = dataset['test'][sample_num]
    
    if do_have_lotta_output_from_all_dialogs_summaries_2:
        print("="*75)
        print(f"dialogue: \n{this_sample['dialogue']}\n---------------")
    ##endof:  if do_have_lotta_output_from_all_dialogs_summaries_2
    
    ground_tat_summary = this_sample['summary']
    res_tat = tat_summarizer(this_sample['dialogue'])
    res_tat_summary = res_tat[0]['summary_text']
    
    if do_have_lotta_output_from_all_dialogs_summaries_2:
        print("-"*70)
        print(f"human-genratd summary:\n{ground_tat_summary}")
        print("-"*70)
        print(f"flan-t5-small summary:\n{res_tat_summary}")
        print("-"*70)
    ##endof:  if do_have_lotta_output_from_all_dialogs_summaries_2

    tat_sample_dialog_list.append(this_sample['dialogue'])
    reference_tat_list.append(ground_tat_summary)
    prediction_tat_list.append(res_tat_summary)
##endof:  for sample_num in range(len(dataset['test']))

tat_toc = timeit.default_timer()

tat_duration = tat_toc = tat_tic

print( "Getting things ready for scoring (after training)")
print(f"took {tat_toc - tat_tic:0.4f} seconds.")

tat_time_str = format_timespan(tat_duration)

print(f"which equates to {tat_time_str}")

rouge = load_metric('rouge', trust_remote_code=True)

results_tat = compute_google_rouge_score(
                  predictions=prediction_tat_list,
                  references=reference_tat_list,
                  use_aggregator=True
)

# >>> print(list(results_tat.keys()))
# ['rouge1', 'rouge2', 'rougeL', 'rougeLsum']

objects_to_pickle.append(tat_sample_dialog_list)
objects_to_pickle_var_names.append('tat_sample_dialog_list')
objects_to_pickle.append(prediction_tat_list)
objects_to_pickle_var_names.append('prediction_tat_list')
objects_to_pickle.append(reference_tat_list)
objects_to_pickle_var_names.append('reference_tat_list')
objects_to_pickle.append(results_tat)
objects_to_pickle_var_names.append('results_tat')

#incremental
objects_to_pickle_03.append(tat_sample_dialog_list)
objects_to_pickle_var_names_03.append('tat_sample_dialog_list')
objects_to_pickle_03.append(prediction_tat_list)
objects_to_pickle_var_names_03.append('prediction_tat_list')
objects_to_pickle_03.append(reference_tat_list)
objects_to_pickle_var_names_03.append('reference_tat_list')
objects_to_pickle_03.append(results_tat)
objects_to_pickle_var_names_03.append('results_tat')

In [ ]:
##  Haven't tried this, because the logging seemed easier,
##+ and the logging worked
# os.environ("TRANSFORMERS_NO_ADVISORY_WARNINGS") = init_t_n_a_w

logging.set_verbosity(init_log_verbosity)

In [ ]:
print_rouge_scores(results_tat, "TEST AFTER TRAINING")

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

### Any comparison

In [ ]:
# any comparison code

<br/><br/>
<hr/>

## A Saving Checkpoint - 3

In [ ]:
#incremental
pickle_filename_after_lora_test = "lora_flan_t5_cpu_objects_-_03_after_lora_test.pkl"
pickle_filename_03 = pickle_filename_before_training

objects_to_pickle_var_names_03.append('objects_to_pickle_var_names_03')
objects_to_pickle_03.append(objects_to_pickle_var_names_03)

with open(pickle_filename_03, 'wb') as pfh:
    pickle.dump(objects_to_pickle_03 , pfh)
##endof:  with open ... as pfh # (pickle file handle)

## Now, ALL Pickle things to pickle save

In [ ]:
objects_to_pickle_var_names.append('objects_to_pickle_var_names')

objects_to_pickle.append(objects_to_pickle_var_names)

with open(pickle_filename, 'wb') as pfh:
    pickle.dump(objects_to_pickle , pfh)
##endof:  with open ... as pfh # (pickle file handle)

In [ ]:
# # Don't need this again
!powershell -c (Get-Date -UFormat "%s_%Y-%m-%dT%H%M%S%Z00") -replace '[.][0-9]*_', '_'

Output was:

`timestamp`

<hr/>

## Notes Looking Forward to LoRA on RWKV - minimize

Hugging Face Community, seems to have a good portion of their models

https://huggingface.co/RWKV

https://web.archive.org/web/20240530232509/https://huggingface.co/RWKV

<br/>

GitHub has even more versions/models, including the `v4-neo` that
I think will be important (the LoRA project)

https://github.com/BlinkDL/RWKV-LM/tree/main

https://web.archive.org/web/20240530232637/https://github.com/BlinkDL/RWKV-LM/tree/main

<br/>

The main RWKV website (?!)

https://www.rwkv.com/

https://web.archive.org/web/20240529120904/https://www.rwkv.com/

<br/>
<br/>

GOOD STUFF. A project doing LoRA with RWKV

https://github.com/Blealtan/RWKV-LM-LoRA/

https://web.archive.org/web/20240530232823/https://github.com/Blealtan/RWKV-LM-LoRA

<br/>
<br/>

The official blog, I guess, with some good coding examples

https://huggingface.co/blog/rwkv

https://web.archive.org/web/20240530233025/https://huggingface.co/blog/rwkv

It includes something that's similar to what I'm doing here in the
`First_Full_LoRA_Trial_with_Transformer_Again.ipynb` tutorial, etc.

```
from transformers import AutoTokenizer, AutoModelForCausalLM

model_id = "RWKV/rwkv-raven-1b5"

model = AutoModelForCausalLM.from_pretrained(model_id).to(0)
tokenizer = AutoTokenizer.from_pretrained(model_id)
```

The `AutoModelForCausalLM` is the same as the tutorial I'm following,
but I don't know what the `.to(0)` is for.

Really quickly, also looking at

https://huggingface.co/RWKV/rwkv-4-world-7b

https://web.archive.org/web/20240530234438/https://huggingface.co/RWKV/rwkv-4-world-7b

I see an example for CPU.

```
model = AutoModelForCausalLM.from_pretrained(
              "RWKV/rwkv-4-world-7b",
              trust_remote_code=True
).to(torch.float32)

tokenizer = AutoTokenizer.from_pretrained(
              "RWKV/rwkv-4-world-7b",
              trust_remote_code=True)
```

<br/><br/>

(Old version? Unofficial, it seems)

https://huggingface.co/docs/transformers/en/model_doc/rwkv

https://web.archive.org/web/20240530232341/https://huggingface.co/docs/transformers/en/model_doc/rwkv